# Operations problems with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

PowerSimulations.jl supports the construction and solution of relaxed optimal power flow
problems (Operations Problems). Operations problems form the fundamental
building blocks for [sequential simulations](../../notebook/PowerSimulations_examples/sequential_simulations.ipynb).
This example shows how to specify and customize a the mathematics that will be applied to the data with
an `OperationsProblemTemplate`, build and execute an `OperationsProblem`, and access the results.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using InfrastructureSystems
const IS = InfrastructureSystems
using PowerSystems
const PSY = PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using JuMP
using Cbc #solver

### Data
This data depends upon the [RTS-GMLC](https://github.com/grid-mod/rts-gmlc) dataset. Let's
download and extract the data.

In [5]:
rts_dir = SIIPExamples.download("https://github.com/GridMod/RTS-GMLC")
rts_src_dir = joinpath(rts_dir, "RTS_Data", "SourceData")
rts_siip_dir = joinpath(rts_dir, "RTS_Data", "FormattedData", "SIIP");

### Create a `System` from RTS-GMLC data just like we did in the [parsing tabular data example.](../../notebook/PowerSystems_examples/parse_tabulardata.jl)

In [6]:
rawsys = PSY.PowerSystemTableData(rts_src_dir,
                                  100.0,
                                  joinpath(rts_siip_dir,"user_descriptors.yaml"),
                                  timeseries_metadata_file = joinpath(rts_siip_dir,"timeseries_pointers.json"),
                                  generator_mapping_file = joinpath(rts_siip_dir,"generator_mapping.yaml"));

sys = System(rawsys; forecast_resolution = Dates.Hour(1));

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/wHSk9/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/wHSk9/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/wHSk9/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/wHSk9/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/wHSk9/src/parsers/power_system_table_data.jl:148
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/wHSk9/src/parsers/power_system_table_data.jl:153
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/

## Change the device type of inter-regional ties from Line to MonitoredLines
to enforce flow limits between regions

In [7]:
regional_lines = ["AB1", "AB2", "AB3", "CA-1", "CB-1"];
for name in regional_lines
    line = get_components_by_name(ACBranch, sys, name)
    convert_component!(MonitoredLine, line[1], sys)
end

For now, let's just choose a standard DCOPF formulation.

In [8]:
uc_template = template_unit_commitment(network = DCPPowerModel)


Operations Problem Specification

  transmission:  DCPPowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroFixed
      Generators:
        device_type = ThermalStandard
        formulation = ThermalBasicUnitCommitment
      DistRE:
        device_type = RenewableFix
        formulation = RenewableFixed
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchReservoirFlow
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
 

## `OperationsProblem`
Now that we have a `System` and an `OperationsProblemTemplate`, we can put the two together
to create an `OperationsProblem` that we solve.

### Optimizer
It's most convenient to define an optimizer instance upfront and pass it into the
`OperationsProblem` constructor. For this example, we can use the free Cbc solver with a
relatively relaxed MIP gap (`ratioGap`) setting to improve speed.

In [9]:
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Build an `OperationsProblem`
The construction of an `OperationsProblem` essentially applies an `OperationsProblemTemplate`
to `System` data to create a JuMP model.

In [10]:
op_problem = OperationsProblem(GenericOpProblem,
                               uc_template,
                               sys;
                               optimizer = solver,
                               horizon = 12,
                               slack_variables=true
)

┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/PKJGB/src/devices_models/device_constructors/common/constructor_validations.jl:4
┌ Warning: The data doesn't include devices of type HydroEnergyReservoir, consider changing the device models
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/PKJGB/src/devices_models/device_constructors/common/constructor_validations.jl:4



Operations Problem Specification

  transmission:  DCPPowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroFixed
      Generators:
        device_type = ThermalStandard
        formulation = ThermalBasicUnitCommitment
      DistRE:
        device_type = RenewableFix
        formulation = RenewableFixed
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchReservoirFlow
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticLine
      DC:
 

And solve it ... (the initial conditions for the RTS in this problem are infeasible)

In [11]:
solve!(op_problem)

┌ Warning: SolveTime() property not supported by COIN Branch-and-Cut (Cbc)
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/PKJGB/src/core/operations_problem.jl:494


,322_CT_6,321_CC_1,202_STEAM_3,315_STEAM_1,223_CT_4,123_STEAM_2,213_CT_1,223_CT_6
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
2,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
3,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
4,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
5,0.0,0.0,0.3,0.0,0.0,0.62,0.0,0.0
6,0.0,0.0,0.3,0.0,0.0,0.93,0.0,0.0
7,0.0,0.0,0.606667,0.0,0.0,1.01118,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.93,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.62,0.0,0.0


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*